In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import cv2

from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import pathlib

# Dataset Creation

In [ ]:
data_dir = pathlib.Path('dataset/') 

SEED_VALUE = 7
SHUFFLE_MODE = True
VAL_SPLIT = 0.3

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
directory = data_dir,
labels = 'inferred',
label_mode = 'categorical',
image_size = IMG_SIZE,
validation_split = VAL_SPLIT,
subset = "training",
seed = SEED_VALUE,
shuffle = SHUFFLE_MODE)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
directory = data_dir,
labels = 'inferred',
label_mode = 'categorical',
image_size = IMG_SIZE,
validation_split = VAL_SPLIT,
subset = "validation",
seed = SEED_VALUE,
shuffle = SHUFFLE_MODE)

In [ ]:
class_names = train_dataset.class_names
num_classes = len(class_names)
print(class_names)

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomBrightness(0.2),
  tf.keras.layers.RandomContrast(0.2)
])

In [ ]:
for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

# Model Creation

In [ ]:
#preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
preprocess_input = tf.keras.applications.vgg16.preprocess_input
rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)

base_model = tf.keras.applications.VGG16(
    include_top=False,
    weights='imagenet'
)

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
base_model.trainable = False
base_model.summary()

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = tf.keras.layers.Lambda(lambda x: preprocess_input(x), output_shape=(224, 224, 3))(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

# Training

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
initial_epochs = 5
loss0, accuracy0 = model.evaluate(validation_dataset)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))


In [ ]:
history = model.fit(train_dataset,
                    epochs=2,
                    validation_data=validation_dataset)

In [ ]:
model.save("fr_model.keras")

# Model verification

In [ ]:
model.load_weights("fr_model.keras")

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

In [ ]:
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(label_batch, axis=1)

print('Predictions:\n', predicted_classes)
print('Labels:\n', true_classes)

plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image_batch[i].astype("uint8"))
    plt.title(class_names[predicted_classes[i]])
    plt.axis("off")

In [ ]:
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
single_image = image_batch[0:1]
predictions = model.predict(single_image)
predicted_class_index = np.argmax(predictions, axis=1)
predicted_class_name = class_names[predicted_class_index[0]]

plt.figure(figsize=(4, 4))
plt.imshow(single_image[0].astype("uint8")) 
plt.title(f'Predicted: {predicted_class_name}')
plt.axis('off')
plt.show()

In [ ]:
def prediction(file_path):
    img = tf.keras.utils.load_img(
    file_path, target_size=(224, 224)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    
    if predictions.shape[0] == 1:
        predictions = predictions[0]  

    zipped_results = list(zip(class_names, predictions))

    for class_name, prediction in zipped_results:
        print(f'Class: {class_name}, Prediction: {prediction}')

    print(
        "This image most likely belongs to {}."
        .format(class_names[np.argmax(score)])
    )

In [ ]:
josh_path = '2a0a8ccc-7c1a-11ee-9b35-244bfe0536bf.jpg'
josh_img = prediction(josh_path)

sara_path = '6c5e4a9d-e809-11ee-9b1b-6c6a77a009ff.jpg'
sara_img = prediction(sara_path)

jad_path = '8bcc6c48-e7d9-11ee-b856-7a8da97ec93a.jpg'
jad_img = prediction(jad_path)

In [ ]:
rob_path = 'IMG_1200.jpeg'
rob_img = prediction(rob_path)

# Make predictions

In [ ]:
update_mode = False

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def predict_image(image):
    img_array = np.expand_dims(image, axis=0)
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    
    if predictions.shape[0] == 1:
        predictions = predictions[0]  

    zipped_results = list(zip(class_names, predictions))

    for class_name, prediction in zipped_results:
        print(f'Class: {class_name}, Prediction: {prediction}')
    
    print(
        "This image most likely belongs to {}."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )
    return img_array, np.argmax(score)

def update_model(img_array, correct_label):
    model.fit(img_array, tf.keras.utils.to_categorical([correct_label], num_classes=len(class_names)), verbose=0)

print("Press 'c' to capture the image and predict, 'u' to toggle update mode, or 'q' to quit.")
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Can't receive frame. Exiting ...")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    cv2.imshow('Webcam - Press "c" to capture and predict, "u" to toggle update, "q" to quit', frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('c'):
        if faces is not None:
            for (x, y, w, h) in faces:
                face = frame[y:y+h, x:x+w]
                face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                face_resized = cv2.resize(face_rgb, (224, 224))
                face_preprocessed = tf.keras.applications.vgg16.preprocess_input(face_resized)
                img_array, predicted_label = predict_image(face_preprocessed)
                print("Press a number key to select a label")
                break 

    elif key == ord('u'):
        update_mode = not update_mode
        print(f"Update mode set to {'on' if update_mode else 'off'}.")
        print(f"Classes: {class_names}")

    elif ord('0') <= key <= ord('9'):
        if update_mode:
            correct_label = key - ord('0')
            if correct_label < len(class_names):
                update_model(img_array, correct_label)
                print(f"Classes: {class_names}")
                print(f"Model updated with label: {class_names[correct_label]}")
            else:
                print("Invalid class label.")

    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Get faces from dataset

In [ ]:
import cv2
import os
import glob

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

dataset_dir = 'dataset/Sara/'
processed_dir = 'dataset_new/Sara/'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

for filepath in glob.glob(os.path.join(dataset_dir, '*.jpg')):
    filename = os.path.basename(filepath)
    img = cv2.imread(filepath)
    if img is not None:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        for i, (x, y, w, h) in enumerate(faces):
            face = img[y:y+h, x:x+w]
            resized_face = cv2.resize(face, (224, 224))

            save_path = os.path.join(processed_dir, f"{filename[:-4]}_face_{i}.jpg")
            cv2.imwrite(save_path, resized_face)
            print(f"Processed and saved face: {save_path}")